In [ ]:
from openai import OpenAI
import yaml
import os
import json
from prompt import *
# from generate import *
# from t2i import *
from tts import *
from generate import generate_and_parse_story # 只导入我们需要的统一函数
from image_generator import process_story_for_images # 导入图片处理函数

In [ ]:
with open("config.yaml", "r", encoding='utf-8') as file:
    config = yaml.safe_load(file)

In [ ]:
prompt = generate_prompt(config=config)
print("--- Generating story with the following prompt ---")
print(prompt)